# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
house_df = pd.read_csv('./cal_housing_clean.csv')

In [4]:
house_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [8]:
house_df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [6]:
house_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [10]:
x_data = house_df.drop('medianHouseValue', axis=1)

In [13]:
y_data = house_df['medianHouseValue']
y_data.head()

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: medianHouseValue, dtype: float64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=101)

In [25]:
print(X_train.shape)
print(X_test.shape)

(13828, 6)
(6812, 6)


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [26]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler()

In [28]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [30]:
X_train = pd.DataFrame(data=scaler.transform(X_train), columns=X_train.columns, index=X_train.index)

In [31]:
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [32]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16624,0.509804,0.072766,0.110180,0.032455,0.087486,0.146246
14848,0.352941,0.092400,0.167132,0.109372,0.180398,0.056834
11081,0.647059,0.050918,0.051210,0.029409,0.053774,0.213107
930,0.431373,0.080574,0.082402,0.050520,0.084526,0.271431
5889,0.764706,0.077980,0.117163,0.041557,0.114126,0.211294


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [33]:
X_train.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

In [34]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [35]:
feature_cols = [age, rooms, bedrooms, population, households, income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [36]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [38]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa3b7df6160>, '_log_step_count_steps': 100, '_eval_distribute': None, '_save_checkpoints_steps': None, '_task_id': 0, '_experimental_max_worker_delay_secs': None, '_is_chief': True, '_service': None, '_num_ps_replicas': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_summary_steps': 100, '_train_distribute': None, '_protocol': None, '_device_fn': None, '_experimental_distribute': None, '_model_dir': '/var/folders/0t/3d36yk2909g3l_5vhfrmv5gr0000gn/T/tmpxik1dusy', '_task_type': 'worker', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_master': '', '_session_creation_timeout_secs': 7200, '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_tf_random_seed': None}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [59]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/0t/3d36yk2909g3l_5vhfrmv5gr0000gn/T/tmpxik1dusy/model.ckpt-26000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 26000 into /var/folders/0t/3d36yk2909g3l_5vhfrmv5gr0000gn/T/tmpxik1dusy/model.ckpt.
INFO:tensorflow:loss = 96026240000.0, step = 26001
INFO:tensorflow:global_step/sec: 251.778
INFO:tensorflow:loss = 81157560000.0, step = 26101 (0.399 sec)
INFO:tensorflow:global_step/sec: 293.382
INFO:tensorflow:loss = 116195844000.0, step = 26201 (0.340 sec)
INFO:tensorflow:global_step/sec: 282.723
INFO:tensorflow:loss = 136027865000.0, step = 26301 (0.354 sec)
INFO:tensorflow:global_step/sec: 313.628
INFO:tensorflow:loss = 162329560000.0, step = 26401 (0.319 sec)
INFO:tensorflow:global_step/sec: 314.771
INFO:tens

INFO:tensorflow:global_step/sec: 299.838
INFO:tensorflow:loss = 58077397000.0, step = 33501 (0.333 sec)
INFO:tensorflow:global_step/sec: 313.067
INFO:tensorflow:loss = 43685315000.0, step = 33601 (0.319 sec)
INFO:tensorflow:global_step/sec: 301.46
INFO:tensorflow:loss = 225716700000.0, step = 33701 (0.332 sec)
INFO:tensorflow:global_step/sec: 312.558
INFO:tensorflow:loss = 48014713000.0, step = 33801 (0.320 sec)
INFO:tensorflow:global_step/sec: 325.128
INFO:tensorflow:loss = 60974158000.0, step = 33901 (0.310 sec)
INFO:tensorflow:global_step/sec: 294.319
INFO:tensorflow:loss = 110434200000.0, step = 34001 (0.338 sec)
INFO:tensorflow:global_step/sec: 298.443
INFO:tensorflow:loss = 99492364000.0, step = 34101 (0.335 sec)
INFO:tensorflow:global_step/sec: 299.113
INFO:tensorflow:loss = 61603963000.0, step = 34201 (0.335 sec)
INFO:tensorflow:global_step/sec: 296.385
INFO:tensorflow:loss = 56482296000.0, step = 34301 (0.337 sec)
INFO:tensorflow:global_step/sec: 322.194
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 324.548
INFO:tensorflow:loss = 148933280000.0, step = 41401 (0.314 sec)
INFO:tensorflow:global_step/sec: 308.938
INFO:tensorflow:loss = 88197900000.0, step = 41501 (0.318 sec)
INFO:tensorflow:global_step/sec: 321.423
INFO:tensorflow:loss = 88411580000.0, step = 41601 (0.316 sec)
INFO:tensorflow:global_step/sec: 307.039
INFO:tensorflow:loss = 150462900000.0, step = 41701 (0.321 sec)
INFO:tensorflow:global_step/sec: 297.294
INFO:tensorflow:loss = 61022030000.0, step = 41801 (0.339 sec)
INFO:tensorflow:global_step/sec: 315.641
INFO:tensorflow:loss = 40137330000.0, step = 41901 (0.315 sec)
INFO:tensorflow:global_step/sec: 296.367
INFO:tensorflow:loss = 93365480000.0, step = 42001 (0.337 sec)
INFO:tensorflow:global_step/sec: 293.302
INFO:tensorflow:loss = 63486296000.0, step = 42101 (0.343 sec)
INFO:tensorflow:global_step/sec: 313.091
INFO:tensorflow:loss = 48924934000.0, step = 42201 (0.317 sec)
INFO:tensorflow:global_step/sec: 299.746
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 315.549
INFO:tensorflow:loss = 110604040000.0, step = 49301 (0.324 sec)
INFO:tensorflow:global_step/sec: 295.121
INFO:tensorflow:loss = 61481247000.0, step = 49401 (0.332 sec)
INFO:tensorflow:global_step/sec: 300.133
INFO:tensorflow:loss = 175061140000.0, step = 49501 (0.334 sec)
INFO:tensorflow:global_step/sec: 299.031
INFO:tensorflow:loss = 106473470000.0, step = 49601 (0.334 sec)
INFO:tensorflow:global_step/sec: 318.153
INFO:tensorflow:loss = 27745290000.0, step = 49701 (0.314 sec)
INFO:tensorflow:global_step/sec: 300.628
INFO:tensorflow:loss = 134166080000.0, step = 49801 (0.332 sec)
INFO:tensorflow:global_step/sec: 248.766
INFO:tensorflow:loss = 38531070000.0, step = 49901 (0.402 sec)
INFO:tensorflow:global_step/sec: 226.604
INFO:tensorflow:loss = 71215050000.0, step = 50001 (0.454 sec)
INFO:tensorflow:global_step/sec: 171.673
INFO:tensorflow:loss = 52005786000.0, step = 50101 (0.578 sec)
INFO:tensorflow:global_step/sec: 163.284
INFO:tensorflow:los

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [60]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [61]:
pred_gen = model.predict(predict_input_func)

In [62]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/0t/3d36yk2909g3l_5vhfrmv5gr0000gn/T/tmpxik1dusy/model.ckpt-51000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [63]:
final_pred = [pred['predictions'] for pred in predictions]

In [64]:
from sklearn.metrics import mean_squared_error

In [65]:
Ideal = 97921.93181985477

In [66]:
mean_squared_error(y_test,final_pred)**0.5

91519.68721471839

# Great Job!